## This code illustrates how to get S&P500 index constituents and their identifiers from CRSP and Compustat
- Researchers used to be able to extract index membership information from Compustat's "comp.idxcst_his" data. Now that S&P pulled this piece of data off from WRDS platform, we have to turn to CRSP to get S&P500 Index membership data. 
- Unfortunately, there is no easy way to uncover constituents info for the other indices covered by "comp.idxcst_his".

In [25]:
##########################################
# S&P 500 Index Constituents             #
# Qingyi (Freda) Song Drechsler          #
# Date: October 2020                     #
##########################################

import pandas as pd
import wrds

### Step 1: Connect to WRDS

In [26]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

Loading library list...
Done


### Step 2: Get SP500 Index Membership from CRSP
- I opt for the monthly frequency of the data, but one can choose to work with crsp.dsp500list if more precise date range is needed.

In [27]:
sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start
                        and b.date <= a.ending
                        and b.date >= '01/01/1980'
                        and b.date <= '12/31/2017'
                        order by date;
                    """, date_cols=['start', 'ending', 'date'])

In [28]:
sp500.head()

,permno,start,ending,date,ret
0,25056.0,1957-03-01,1997-08-05,1980-01-31,0.034335
1,24678.0,1957-03-01,2000-06-20,1980-01-31,0.087500
2,44505.0,1968-01-18,1982-08-11,1980-01-31,0.265116
3,18315.0,1957-03-01,1982-09-15,1980-01-31,0.116667
4,38973.0,1963-03-25,1998-01-29,1980-01-31,-0.050505


### Step 3: Add Other Company Identifiers from CRSP.MSENAMES
- You don't need this step if only PERMNO is required
- This step aims to add TICKER, SHRCD, EXCHCD and etc. 

In [29]:
# Add Other Descriptive Variables

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.Timestamp('2017-12-31'))

In [30]:
mse.sample(5)

,comnam,ncusip,namedt,nameendt,permno,shrcd,exchcd,hsiccd,ticker
15974,ISHARES TRUST,46435G78,2016-11-03,2017-12-25,15682.0,73.0,4.0,6726.0,IBMJ
15277,ASSOCIATED MEDICAL DEVICES INC,04571010,1982-11-01,1984-01-26,15393.0,10.0,3.0,3823.0,ASMD
79452,AUBURN NATIONAL BANCORP,05047310,2004-06-10,2006-06-29,82251.0,11.0,3.0,6020.0,AUBN
109792,UR ENERGY INC,91688R10,2008-07-24,2008-07-24,92747.0,12.0,2.0,1094.0,URG
31055,ASHLAND OIL & REFNG CO,04453710,1968-01-02,1970-02-01,24272.0,11.0,1.0,5169.0,ASH


In [31]:
# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]
sp500_full.sample(5)

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker
1469675,78877.0,2006-03-03,2018-03-16,2015-03-31,-0.151079,CHESAPEAKE ENERGY CORP,16516710,2004-06-10,2016-12-26,11.0,1.0,1311.0,CHK
378607,26382.0,1980-01-17,2000-12-21,1990-03-30,-0.001516,FLUOR CORP,34386110,1976-01-02,2000-11-30,11.0,1.0,1221.0,FLR
323887,52695.0,1981-05-14,2022-12-30,1988-10-31,0.006682,GRAINGER W W INC,38480210,1975-07-29,2001-02-28,11.0,1.0,5085.0,GWW
1078091,49905.0,1984-10-25,2008-12-01,2006-07-31,-0.046141,LIZ CLAIBORNE INC,53932010,2006-06-30,2011-04-03,11.0,1.0,5651.0,LIZ
414073,48397.0,1976-07-01,2001-08-29,1991-01-31,0.065041,AMERICAN GENERAL CORP,02635110,1980-07-01,1999-08-01,11.0,1.0,6311.0,AGC


### Step 4: Add Compustat Identifiers
- Link with Compustat's GVKEY and IID if need to work with fundamental data
- Linkage is done through crsp.ccmxpf_linktable

In [32]:
# Linking with Compustat through CCM

ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.Timestamp('2017-12-31'))

In [33]:
# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]
sp500ccm.sample(5)

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker,gvkey,iid,linktype,linkprim,linkdt,linkenddt
286735,86111.0,2007-11-19,2022-12-30,2013-04-30,0.095294,AMERICAN TOWER CORP NEW,03027X10,2012-01-03,2013-06-18,18.0,1.0,6798.0,AMT,105365,01,LC,P,1998-06-05,2017-12-31
116276,21514.0,1957-03-01,1999-10-25,1992-04-30,0.188525,SONAT INC,83541510,1982-01-05,1999-01-31,11.0,1.0,4923.0,SNT,009810,01,LU,P,1962-01-31,1999-10-29
145883,23077.0,1957-03-01,2013-06-06,1995-08-31,-0.023055,HEINZ H J CO,42307410,1968-01-02,2001-02-28,11.0,1.0,2033.0,HNZ,005568,01,LC,P,1962-01-31,2013-05-31
275174,82298.0,2009-02-26,2016-09-30,2011-10-31,0.213281,DIAMOND OFFSHORE DRILLING INC,25271C10,2004-06-10,2020-04-24,11.0,1.0,1381.0,DO,061409,01,LC,P,1995-10-11,2020-04-24
159958,75034.0,1987-04-09,2022-12-30,1997-04-30,-0.100977,BAKER HUGHES INC,05722410,1987-04-24,2001-08-23,11.0,1.0,3533.0,BHI,001976,01,LC,P,1987-04-24,2017-07-04


In [34]:
# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', \
                                  'linktype', 'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip', 'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]
sp500ccm.sample(5)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
215678,2004-01-30,89813.0,MEDCO HEALTH SOLUTIONS INC,58405U10,11.0,1.0,5912.0,MHS,007186,02,2003-08-20,2012-04-03,0.084142
19147,1981-11-30,24109.0,AMERICAN ELECTRIC POWER CO INC,02553710,11.0,1.0,4911.0,AEP,001440,01,1951-09-06,2022-12-30,0.055704
73514,1987-07-31,45671.0,SCIENTIFIC ATLANTA INC,80865510,11.0,1.0,3663.0,SFA,009483,01,1981-02-26,2006-02-24,-0.089351
219470,2004-07-30,87842.0,METLIFE INC,59156R10,11.0,1.0,6311.0,MET,133768,01,2000-12-11,2022-12-30,-0.005021
72188,1987-06-30,17830.0,UNITED TECHNOLOGIES CORP,91301710,11.0,1.0,3724.0,UTX,010983,01,1957-03-01,2022-12-30,0.148352


In [35]:
cnt = sp500ccm.groupby(['date'])['permno'].count().reset_index().rename(columns={'permno':'npermno'})
cnt.sample(4)

,date,npermno
260,2001-09-28,500
446,2017-03-31,501
0,1980-01-31,502
268,2002-05-31,500


In [38]:
sp500ccm.head(3)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
1,1980-01-31,25056.0,ENSERCH CORP,29356710,11.0,1.0,4924.0,ENS,004393,01,1957-03-01,1997-08-05,0.034335
3,1980-01-31,24678.0,WARNER LAMBERT CO,93448810,11.0,1.0,2834.0,WLA,011288,01,1957-03-01,2000-06-20,0.087500
4,1980-01-31,44505.0,GIDDINGS & LEWIS INC,37504610,10.0,1.0,3541.0,GID,005163,01,1968-01-18,1982-08-11,0.265116


In [39]:
sp500ccm = sp500ccm.rename(columns={'start': 'from', 'ending': 'thru'})

In [41]:
sp500ccm['spmi']="10"
sp500ccm.head()

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,from,thru,ret,spmi
1,1980-01-31,25056.0,ENSERCH CORP,29356710,11.0,1.0,4924.0,ENS,004393,01,1957-03-01,1997-08-05,0.034335,10
3,1980-01-31,24678.0,WARNER LAMBERT CO,93448810,11.0,1.0,2834.0,WLA,011288,01,1957-03-01,2000-06-20,0.087500,10
4,1980-01-31,44505.0,GIDDINGS & LEWIS INC,37504610,10.0,1.0,3541.0,GID,005163,01,1968-01-18,1982-08-11,0.265116,10
6,1980-01-31,18315.0,ALPHA PORTLAND INDS INC,02077110,11.0,1.0,1629.0,APC,009766,02,1957-03-01,1982-09-15,0.116667,10
7,1980-01-31,38973.0,I C INDUSTRIES INC,44926810,11.0,1.0,2086.0,ICX,005824,01,1963-03-25,1998-01-29,-0.050505,10


In [42]:
sp500ccm.to_stata("Index Constituents.dta")